## Map Mutations to Protein-Protein and Protein-Nucleic Acid Interfaces
Here we find and visualize the mutations that occur around protein-protein and protein-nucleic acid interfaces.

In [1]:
# Disable Numba: temporary workaround for https://github.com/sbl-sdsc/mmtf-pyspark/issues/288
import os
os.environ['NUMBA_DISABLE_JIT'] = "1"

In [2]:
from pyspark.sql import SparkSession
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.interactions import InteractionExtractor, InteractionFilter
from ipywidgets import interact, IntSlider, FloatSlider, SelectMultiple
import py3Dmol
import pandas as pd

In [3]:
# Initialize Spark
spark = SparkSession.builder.appName("2-MapToPolymerInteractions").getOrCreate()

In [4]:
# Enable Arrow-based columnar data transfers between Spark and Pandas dataframes
# Commented out for deployment on Pangeo Binder since it causes an out of memory error
# Exceeds spark.driver.maxResultSize (1024.0 MB)
#spark.conf.set("spark.sql.execution.arrow.enabled", "true")

#### Input parameters

In [5]:
distance_cutoff = 8 # distance cutoff for finding and visualizing interactions
input_file_name = 'mutations3d.csv' # mutations mapped to 3D protein structures
output_file_name = 'mutations3d_polymer_inter.csv' # mutations mapped to polymer interactions

## Read mutation file created in the previous step

In [6]:
pd.set_option('display.max_columns', None)  # show all columns
df = pd.read_csv(input_file_name)
df['pdbPosition'] = df['pdbPosition'].astype('str') # PDB residue numbers must be string to handle insertion codes
df.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,annotation,scale,color
0,0.026709,"Spike protein S2', water",Spike glycoprotein,2.47,6LVN.A,6LVN,1169,2,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d
1,0.027494,"Spike protein S2', water",Spike glycoprotein,2.47,6LVN.B,6LVN,1169,2,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d
2,0.026709,"Spike protein S2', water",Spike glycoprotein,2.47,6LVN.D,6LVN,1169,2,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d
3,0.273370,"Spike glycoprotein, 2-acetamido-2-deoxy-beta-D...",Spike glycoprotein,3.00,6XRA.A,6XRA,1169,1169,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d
4,0.273370,"Spike glycoprotein, 2-acetamido-2-deoxy-beta-D...",Spike glycoprotein,3.00,6XRA.B,6XRA,1169,1169,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d


## Create a list of unique PDB Ids

In [7]:
pdb_ids = list(df.structureId.drop_duplicates())
print("PDB Ids:", pdb_ids)

PDB Ids: ['6LVN', '6XRA', '6LXT', '6VSB', '6VXX', '6VYB', '6WPS', '6WPT', '6X29', '6X2A', '6X2B', '6X2C', '6X6P', '6X79', '6XCM', '6XCN', '6XEY', '6XF5', '6XF6', '6XKL', '6XLU', '6XM0', '6XM3', '6XM4', '6XM5', '6XR8', '6XS6', '6Z43', '6Z97', '6ZB4', '6ZB5', '6ZDH', '6ZGE', '6ZGG', '6ZGH', '6ZGI', '6ZHD', '6ZOW', '6ZOX', '6ZOY', '6ZOZ', '6ZP0', '6ZP1', '6ZP2', '6ZP5', '6ZP7', '6ZWV', '6ZXN', '7A25', '7A29', '7A4N', '7A93', '7A94', '7A95', '7A96', '7A97', '7A98', '7AD1', '7BNM', '7BNN', '7BNO', '7BYR', '7C2L', '7CAB', '7CAC', '7CAI', '7CAK', '7CHH', '7CN9', '7CT5', '7CWL', '7CWM', '7CWN', '7CWS', '7CWU', '7CZP', '7CZQ', '7CZR', '7CZS', '7CZT', '7CZU', '7CZV', '7CZW', '7CZX', '7CZY', '7CZZ', '7D00', '7D03', '7D0B', '7D0C', '7D0D', '7DCC', '7DCX', '7DD2', '7DD8', '7DDD', '7DDN', '7DF3', '7DF4', '7DK3', '7DK4', '7DK5', '7DK6', '7DK7', '7DWX', '7DWY', '7DWZ', '7DX0', '7DX1', '7DX2', '7DX3', '7DX5', '7DX6', '7DX7', '7DX8', '7DX9', '7E7B', '7E7D', '7JJI', '7JJJ', '7JV4', '7JV6', '7JVC', '7JW0'

## Find all polymer-polymer interactions
Find groups (residues) that interact with other polymer chains

In [8]:
structures = mmtfReader.download_mmtf_files(pdb_ids)

queryGroupId, queryChainId, and queryGroupNumber specify the residue that interacts with another chain (targetChainId)

In [9]:
interaction_filter = InteractionFilter(distanceCutoff=distance_cutoff)

interactions = InteractionExtractor().get_polymer_interactions(structures, interaction_filter, level='group').toPandas()
interactions.head()

,structureChainId,queryGroupId,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence
0,7CZT.B,SER,A,968,PHE,B,759,758,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
1,7CZT.B,TYR,A,1047,THR,B,887,886,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
2,7CZT.A,SER,L,31,ASP,A,420,419,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
3,7CZT.C,THR,A,874,LEU,C,699,698,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...
4,7CZT.A,GLY,C,1093,TYR,A,904,903,MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSS...


In [10]:
interactions[(interactions['structureChainId'] == '6WPS.A') & (interactions['targetGroupId'] == 'ASN')]

,structureChainId,queryGroupId,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence
1552165,6WPS.A,THR,E,167,ASN,A,354,372,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
1552338,6WPS.A,THR,B,547,ASN,A,978,996,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
1552359,6WPS.A,VAL,B,1129,ASN,A,919,937,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
1552378,6WPS.A,ASN,E,978,ASN,A,540,558,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
1552396,6WPS.A,PHE,B,559,ASN,A,282,300,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
1552423,6WPS.A,THR,L,32,ASN,A,343,361,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
1552471,6WPS.A,TYR,E,200,ASN,A,394,412,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
1552564,6WPS.A,THR,B,549,ASN,A,978,996,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
1552610,6WPS.A,LEU,H,110,ASN,A,343,361,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...
1552674,6WPS.A,GLY,E,757,ASN,A,969,987,MGILPSPGMPALLSLVSLLSVLLMGCVAETGTQCVNLTTRTQLPPA...


## Filter mutations by joining with the interaction data

In [11]:
mt = df.merge(interactions, left_on=['structureChainId','pdbPosition'], right_on=['structureChainId','targetGroupNumber'], how='inner')
mt.head()

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,annotation,scale,color,queryGroupId,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence
0,0.026709,"Spike protein S2', water",Spike glycoprotein,2.47,6LVN.A,6LVN,1169,2,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d,LEU,C,33,ILE,A,2,1,DISGINASVVNIQKEIDRLNEVAKNLNESLIDLQEL
1,0.026709,"Spike protein S2', water",Spike glycoprotein,2.47,6LVN.A,6LVN,1169,2,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d,GLN,C,34,ILE,A,2,1,DISGINASVVNIQKEIDRLNEVAKNLNESLIDLQEL
2,0.027494,"Spike protein S2', water",Spike glycoprotein,2.47,6LVN.B,6LVN,1169,2,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d,LEU,D,33,ILE,B,2,1,DISGINASVVNIQKEIDRLNEVAKNLNESLIDLQEL
3,0.027494,"Spike protein S2', water",Spike glycoprotein,2.47,6LVN.B,6LVN,1169,2,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d,GLN,D,34,ILE,B,2,1,DISGINASVVNIQKEIDRLNEVAKNLNESLIDLQEL
4,0.026709,"Spike protein S2', water",Spike glycoprotein,2.47,6LVN.D,6LVN,1169,2,1169,"S:p.1169I>L(54), S:p.1169I>F(28), S:p.1169I>V(...","1169I>L(54), 1169I>F(28), 1169I>V(16), 1169I>T(6)",0.304365,#fcaa8d,GLN,B,34,ILE,D,2,1,DISGINASVVNIQKEIDRLNEVAKNLNESLIDLQEL


In [12]:
mt[(mt['structureChainId'] == '6WPS.A') & (mt['targetGroupId'] == 'ASN') & (mt['targetGroupId'] == '343')]

,coverage,description,name,resolution,structureChainId,structureId,uniprotPosition,pdbPosition,residue,variationId,annotation,scale,color,queryGroupId,queryChainId,queryGroupNumber,targetGroupId,targetChainId,targetGroupNumber,sequenceIndex,sequence


## Save Mappings

In [13]:
mt.to_csv(output_file_name, index=False)

In [14]:
chains = mt.groupby('structureChainId')
print("Chains:", list(chains.groups.keys()))

Chains: ['6LVN.A', '6LVN.B', '6LVN.C', '6LVN.D', '6LXT.A', '6LXT.B', '6LXT.C', '6LXT.D', '6LXT.E', '6LXT.F', '6LZG.B', '6M0J.E', '6M17.E', '6M17.F', '6VSB.A', '6VSB.B', '6VSB.C', '6VW1.E', '6VW1.F', '6VXX.A', '6VXX.B', '6VXX.C', '6VYB.A', '6VYB.B', '6VYB.C', '6W41.C', '6WPS.A', '6WPS.B', '6WPS.E', '6WPT.A', '6WPT.B', '6WPT.C', '6X29.A', '6X29.B', '6X29.C', '6X2A.A', '6X2A.B', '6X2A.C', '6X2B.A', '6X2B.B', '6X2B.C', '6X2C.A', '6X2C.B', '6X2C.C', '6X6P.A', '6X6P.B', '6X6P.C', '6X79.A', '6X79.B', '6X79.C', '6XC2.A', '6XC2.Z', '6XC3.C', '6XC4.A', '6XC4.Z', '6XC7.A', '6XCM.A', '6XCM.B', '6XCM.C', '6XCN.A', '6XCN.C', '6XCN.E', '6XDG.E', '6XE1.E', '6XEY.A', '6XEY.B', '6XEY.C', '6XF5.A', '6XF5.B', '6XF5.C', '6XF6.A', '6XF6.B', '6XF6.C', '6XKL.A', '6XKL.B', '6XKL.C', '6XKP.A', '6XKP.B', '6XKQ.A', '6XLU.A', '6XLU.B', '6XLU.C', '6XM0.A', '6XM0.B', '6XM0.C', '6XM3.A', '6XM3.B', '6XM3.C', '6XM4.A', '6XM4.B', '6XM4.C', '6XM5.A', '6XM5.B', '6XM5.C', '6XR8.A', '6XR8.B', '6XR8.C', '6XRA.A', '6XRA.B', '

## View mutations grouped by protein chain

Mutations are mapped onto protein chains for available 3D protein structures.

Display options:

|||
|:-|:-|
| *show_bio_assembly* | Toggle display of the biologically relevant quaternary structure |
| *show_surface* | Toggle surface for protein chain |
| *show_short_label* | Toggle display of mutation information<br>{UniProtResidue}{aminoAcid1}>{aminoAcid2}(# observations)<br>Example: 501N>Y(350436)|
| *show_long_label* | Toggle display of mutation information<br>{PDBId}.{chainId}.{PDBResidue}: {geneName}.p{UniProtResidue}{aminoAcid1}>{aminoAcid2}(# observations)<br>Example: 6XDG.E.501: S:p.501N>Y(350436) |
| *size* | Change size of visualization |
| *font* | Change font size of annotations |
| *logFreq* | Change minimum threshold to display mutations based on normalized log of mutation frequency [0.0 - 1.0]|
| *structure* | Move slider to browse through available structures |

#### Example: Move the structure slider to PDB ID:6XDG to see how mutations (e.g., 501N>Y) effect the binding of the Regeneron antibodies

In [27]:
# Setup viewer
def view_grouped_mutations(grouped_df, *args):
    chainIds = list(grouped_df.groups.keys())

    def view3d(show_bio_assembly, show_surface, show_short_label, show_long_label, size, font, logFreq, i):
        group = grouped_df.get_group(chainIds[i])
        
        pdb_id, chain_id = chainIds[i].split('.')
        global viewer1
        viewer1 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer1.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})
 
        # highlight chain of interest in blue
        viewer1.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue'}})
        
        # non-polymer style
        viewer1.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
    
        # don't display water molecules
        viewer1.setStyle({'resn': ['HOH','DOD']}, {})
        
        modified_residues = set()
        rows = group.shape[0]
        for j in range(0, rows):
            if group.iloc[j]['scale'] > logFreq:
                res_num = str(group.iloc[j]['pdbPosition'])
                mod_res = {'resi': res_num, 'chain': chain_id} 
                modified_residues.add(res_num)
            
                # style for mutated residue
                col = group.iloc[j]['color']
                c_col = col + 'Carbon'
                viewer1.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
                viewer1.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}}) 
           
                # style for interacting residues
                surroundings = {'not': {'chain': chain_id}, 'byres': True, 'within': {'distance': distance_cutoff, 'sel': mod_res}}
                viewer1.setStyle(surroundings,{'cartoon':{'color':'orange', 'width': 0.6}})
                viewer1.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})
                # style for interacting waters
                waters = {'resn': ['HOH','DOD']}
                waters.update(surroundings)
                viewer1.addStyle(waters,{'sphere':{'color':'orange', 'radius': 0.5}})
                
                if show_short_label or show_long_label:
                    viewer1.addResLabels(surroundings, {'fontSize':font-2})
            
                # text label
                annotation = group.iloc[j]['annotation']
                variationId = group.iloc[j]['variationId']
                
                if show_short_label:
                    label = annotation
                if show_long_label:
                    label = chain_id + "." + str(res_num) + ": " + variationId
                if show_short_label or show_long_label:
                    viewer1.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory', 'opacity': 1.0}, mod_res)
              
        viewer1.zoomTo({'chain': chain_id})
        viewer1.center({'chain': chain_id})
        
        if show_surface:
             viewer1.addSurface(py3Dmol.SES,{'opacity':0.8,'color':'lightblue'},{'chain': chain_id})
                
        # print header
        resolution = group.iloc[0]['resolution']
        coverage = group.iloc[0]['coverage']
        description = group.iloc[0]['description']
        print(f"PDB Id:{pdb_id}, chain Id:{chain_id}, resolution:{resolution}, sequence coverage:{coverage:.2f}")
        print(description)
                
        return viewer1.show()
       
    f_widget = IntSlider(value=9, min=5, max=20, description='font size', continuous_update=False)
    z_widget = IntSlider(value=750, min=500, max=1200, description='size', continuous_update=False)
    s_widget = IntSlider(min=0, max=len(chainIds)-1, description='structure', continuous_update=False)
    l_widget = FloatSlider(value=0.8, min=0, max=1, step=0.05, description='logFreq:', 
                           continuous_update=False, orientation='horizontal', readout=True, readout_format='.2f')
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_short_label=True, show_long_label=False, 
                    size=z_widget, font=f_widget, logFreq=l_widget, i=s_widget)

def view_image1():
    return viewer1.png()

In [28]:
view_grouped_mutations(chains);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

## View one mutation at a time
Use the slider to view each mutation. Interacting residues within the `distance_cutoff` are rendered as orange sticks.

In [23]:
# Setup viewer
def view_single_mutation(df, distance_cutoff, *args):

    def view3d(show_bio_assembly, show_surface, show_short_label, show_long_label, size, font, i):        
        pdb_id, chain_id = df.iloc[i]['structureChainId'].split('.')
        
        global viewer2
        viewer2 = py3Dmol.view(query='pdb:' + pdb_id, options={'doAssembly': show_bio_assembly}, width=size, height=size)

        # polymer style
        viewer2.setStyle({'cartoon': {'colorscheme': 'chain', 'width': 0.6, 'opacity':0.9}})
        
        # highlight chain of interest in blue
        viewer2.setStyle({'chain': chain_id},{'cartoon': {'color': 'blue', 'opacity':0.7}})
        
        # non-polymer style
        viewer2.setStyle({'hetflag': True}, {'stick':{'radius': 0.3, 'singleBond': False}})
        
        # interacting residue style
        res_num = str(df.iloc[i]['pdbPosition'])
        label = df.iloc[i]['variationId']     
        mod_res = {'resi': res_num, 'chain': chain_id} 
        col = df.iloc[i]['color']
        c_col = col + 'Carbon'
        viewer2.addStyle(mod_res, {'stick':{'colorscheme':c_col, 'radius': 0.2}})
        viewer2.addStyle(mod_res, {'sphere':{'color':col, 'opacity': 0.6}})   
        
        # don't display water molecules (except below for interactions)
        viewer2.setStyle({'resn': ['HOH','DOD']}, {})
            
        # select residues in interacting chains by distance from mutation site (same chain is excluded)
        surroundings = {'not': {'chain': chain_id}, 'byres': True, 'within': {'distance': distance_cutoff, 'sel': mod_res}}
        
        # set style for interacting residues
        viewer2.setStyle(surroundings,{'cartoon':{'color':'orange', 'width': 0.6}})
        viewer2.addStyle(surroundings,{'stick':{'colorscheme':'orangeCarbon', 'radius': 0.15}})
    
        # set style for interacting waters
        waters = {'resn': ['HOH','DOD']}
        waters.update(surroundings)
        viewer2.addStyle(waters,{'sphere':{'color':'orange', 'radius': 0.5}})
        
        annotation = df.iloc[i]['annotation']
        variationId = df.iloc[i]['variationId']      
        
        if show_short_label:
            label = annotation
        if show_long_label:
            label = chain_id + "." + str(res_num) + ": " + variationId
        if show_short_label or show_long_label:
            viewer2.addResLabels(surroundings, {'fontSize':font-2})
            viewer2.addLabel(label, {'fontSize':font,'fontColor': 'black','backgroundColor':'ivory', 'opacity': 1.0}, mod_res)
            
        viewer2.zoomTo(surroundings)
        viewer2.center(mod_res)
        
        if show_surface:
             viewer2.addSurface(py3Dmol.SES, {'opacity':0.8,'color':'lightblue'}, {'chain': chain_id})
                
        #print header
        print("PDB Id:", pdb_id, "chain Id:" , chain_id, "residue:", res_num, "mutation:", variationId)
        
        # print any specified additional columns from the dataframe
        for a in args:
            print(a + ": " + df.iloc[i][a])
                
        return viewer2.show()
       
    f_widget = IntSlider(value=9, min=5, max=20, description='font size', continuous_update=False)
    z_widget = IntSlider(value=750, min=500, max=1200, description='size', continuous_update=False)
    s_widget = IntSlider(min=0, max=len(df)-1, description='structure', continuous_update=False)
    
    return interact(view3d, show_bio_assembly=False, show_surface=False, show_short_label=True, show_long_label=False, 
                    size=z_widget, font=f_widget, i=s_widget)

def view_image2():
    return viewer2.png()

In [24]:
mt_unique = mt.drop_duplicates(["structureChainId","variationId"])

In [19]:
view_single_mutation(mt_unique, distance_cutoff);

interactive(children=(Checkbox(value=False, description='show_bio_assembly'), Checkbox(value=False, descriptio…

In [20]:
# Shutdown Spark
spark.stop()

## Now run the next step
Map mutations occuring at protein-ligand binding sites: [3-MapToLigandInteractions.ipynb](3-MapToLigandInteractions.ipynb)